In [22]:
import os
import awswrangler as wr
import boto3
import pandas as pd
import numpy as np
import datetime

In [23]:
session = boto3.Session(profile_name='ssm-dev')
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

In [24]:
# AWS directories
s3_root = 's3://crc-data-lake-research/sma/jimmy_woo'
s3_structured_root = 's3://crc-data-lake-structured/tutela'

structured_database = 'crc_data_lake_structured'
research_database = 'crc_data_lake_research'

results_path = '{}/tmp_resuts'.format(s3_root)

# TODO: upload to S3
local_path = "C:\\Users\\hosuk.woo\\Desktop\\350_legget_5km_radius.xlsx"

In [25]:
# sample input
lic_area = "2-006"
band = "600B"
licensee = "Rogers"
block = "A"

In [26]:
# athena call wrapper for convenience
query_builder = lambda x: wr.athena.read_sql_query(
                              x,
                              database=research_database,
                              s3_output=s3_root,
                              keep_files=False,
                              boto3_session=session,
                              ctas_approach=False
                          )

In [27]:
query = """
    WITH license_conditions_pop AS (
        SELECT 
          lc.cond_area AS area,
          lc.min_coverage,
          lc.term_length,
          SUM(g.population_2016) as total_pop
        FROM 
          crc_data_lake_structured.license_conditions AS lc,
          crc_data_lake_structured.ised_grid AS g
        WHERE
          (lc.cond_area = g.tier_2 or lc.cond_area = g.tier_3 or lc.cond_area = g.tier_4)
          AND lc.lic_area = '{0}'
          AND lc.band = '{1}'
        GROUP BY
          lc.cond_area,
          lc.min_coverage,
          lc.term_length
    ),
    
    license_coverage AS (
        SELECT 
          l.area, 
          l.min_coverage,
          l.term_length,
          COALESCE(SUM(r.population_2016), 0) as pop_covered,
          COALESCE(SUM(r.population_2016)/l.total_pop, 0) as perc_covered
        FROM 
          license_conditions_pop AS l
        LEFT JOIN
          (SELECT * 
           FROM 
             trevor_grid_file_test.deployed_grids_sma_600b_test 
           WHERE
             licensee = '{2}'
             AND block='{3}'
             AND service='{1}') AS r
        ON 
          l.area = r.tier_2 
          OR l.area = r.tier_3
          OR l.area = r.tier_4
        GROUP BY 
          l.area, 
          l.min_coverage, 
          l.total_pop,
          l.term_length
    )
    
    SELECT l.*
    FROM license_coverage as l
""".format(lic_area, band, licensee, block)

res = query_builder(query)
res

area  min_coverage  term_length  pop_covered  perc_covered
0   4-071          0.25           20     23982.92      0.557829
1   3-019          0.50           10     76355.41      0.912109
2   4-073          0.40           20     50295.25      0.771633
3   4-053          0.50           20     25570.92      0.398730
4   2-006          0.25            5   2027361.50      0.832291
5   3-016          0.25           10     53397.09      0.470183
6   3-020          0.60           10    148434.35      0.837125
7   3-023          0.60           10    153277.38      0.724995
8   4-055          0.70           20   1321238.52      0.909410
9   4-068          0.50           20     64301.18      0.911261
10  4-075          0.60           20     34218.91      0.745471
11  4-074          0.60           20    119058.47      0.719317
12  4-067          0.70           20     67330.85      0.965610
13  4-057          0.50           20     16455.77      0.524623
14  3-021          0.50           10    131461.73      0.664032
15  4-069          0.50           20     12054.23      0.916660
16  3-022          0.40           10     50295.25      0.771633
17  4-056          0.50           20     36941.32      0.449409
18  3-018          0.60           10     67330.85      0.965610
19  4-070          0.70           20    148434.35      0.837125
20  3-015          0.60           10   1346809.44      0.887821
21  4-072          0.25           20    107478.81      0.693493

In [28]:
query = """
    SELECT *
    FROM
      crc_data_lake_structured.spectrum_area_license
    WHERE
      authorization_status = 'Granted'
      AND band = '{1}'
      AND company_name = '{2}'
      AND block = '{3}'
""".format(lic_area, band, licensee, block)

sal = query_builder(query)
sal = sal[sal['area'].isin(res['area'])]
sal = sal[['licence_number', 'expiry_date', 'area']]
sal

licence_number expiry_date   area
6      010810510    39-05-26  2-006

spectrum_area_license table only contains entry for tier 2 for this current query. assume license number/expiry date are identical for corresponding tier 3s/4s.

In [29]:
#res = pd.merge(res, sal, how='left', on='area')
res['license_number'] = sal['licence_number'].values[0]
res['expiry_date'] = sal['expiry_date'].values[0]
res

area  min_coverage  term_length  pop_covered  perc_covered  \
0   4-071          0.25           20     23982.92      0.557829   
1   3-019          0.50           10     76355.41      0.912109   
2   4-073          0.40           20     50295.25      0.771633   
3   4-053          0.50           20     25570.92      0.398730   
4   2-006          0.25            5   2027361.50      0.832291   
5   3-016          0.25           10     53397.09      0.470183   
6   3-020          0.60           10    148434.35      0.837125   
7   3-023          0.60           10    153277.38      0.724995   
8   4-055          0.70           20   1321238.52      0.909410   
9   4-068          0.50           20     64301.18      0.911261   
10  4-075          0.60           20     34218.91      0.745471   
11  4-074          0.60           20    119058.47      0.719317   
12  4-067          0.70           20     67330.85      0.965610   
13  4-057          0.50           20     16455.77      0.524623   
14  3-021          0.50           10    131461.73      0.664032   
15  4-069          0.50           20     12054.23      0.916660   
16  3-022          0.40           10     50295.25      0.771633   
17  4-056          0.50           20     36941.32      0.449409   
18  3-018          0.60           10     67330.85      0.965610   
19  4-070          0.70           20    148434.35      0.837125   
20  3-015          0.60           10   1346809.44      0.887821   
21  4-072          0.25           20    107478.81      0.693493   

   license_number expiry_date  
0       010810510    39-05-26  
1       010810510    39-05-26  
2       010810510    39-05-26  
3       010810510    39-05-26  
4       010810510    39-05-26  
5       010810510    39-05-26  
6       010810510    39-05-26  
7       010810510    39-05-26  
8       010810510    39-05-26  
9       010810510    39-05-26  
10      010810510    39-05-26  
11      010810510    39-05-26  
12      010810510    39-05-26  
13      010810510    39-05-26  
14      010810510    39-05-26  
15      010810510    39-05-26  
16      010810510    39-05-26  
17      010810510    39-05-26  
18      010810510    39-05-26  
19      010810510    39-05-26  
20      010810510    39-05-26  
21      010810510    39-05-26

In [30]:
query = """
SELECT *
FROM crc_data_lake_structured.corporate_names
WHERE corporate_name_corrected = '{}'
""".format(licensee)

names = query_builder(query)
names

corporate_name corporate_name_corrected  \
0  ROGERS COMMUNICATIONS PARTNERSHIP M. VUJOSEVIC...                   Rogers   
1                  Rogers Communications Partnership                   Rogers   
2                  Rogers Communications Canada Inc.                   Rogers   
3                                FIDO SOLUTIONS INC.                   Rogers   
4      DATA & AUDIO-VISUAL WIRELESS ENTERPRISES INC.                   Rogers   
5      Data & Audio-Visual Enterprises Wireless Inc.                   Rogers   
6      Data & Audio-Visual Wireless Enterprises Inc.                   Rogers   

   new_name  new_variant  
0     False        False  
1     False        False  
2     False        False  
3     False        False  
4     False        False  
5     False        False  
6     False        False

In [32]:
df = pd.read_excel(local_path)
df = df[df['Request Name'].notna()]

# drop suffixes and correct licensee names
df['Request Name Simplified'] = df.apply(lambda row: str(row['Request Name']).split('-')[0], axis=1)
df['Revision'] = df.apply(lambda row: str(row['Request Name']).split('-')[1], axis=1)
df['Licensee Company'] = df.apply(lambda row: names[names['corporate_name'] == row['Licensee Company']]['corporate_name_corrected'], axis=1)

# filter by conditions and select features
df = df[(df['Licensee Company'] == licensee) & (df['TX_RES_LI_CAT'] == band)][['Request Name Simplified', 'Revision', 'License Start Date']]

# multiple entries per licensee number due to multiple antennas/sectors
df = df.drop_duplicates()

df.head()

Request Name Simplified Revision      License Start Date
0               010810510      001  2022-01-25 12:00:00 AM

In [33]:
# obtain start dates
res = pd.merge(res, df, how='left', left_on='license_number', right_on='Request Name Simplified')

# format
res = res.rename(columns={"License Start Date": "start_date", "Revision": "revision"})
res = res[['license_number', 'revision', 'area', 'min_coverage', 'term_length', 'pop_covered', 'perc_covered', 'start_date', 'expiry_date']]
res['expiry_date'] = pd.to_datetime(res['expiry_date'])
res['start_date'] = pd.to_datetime(res['start_date'])
res

license_number revision   area  min_coverage  term_length  pop_covered  \
0       010810510      001  4-071          0.25           20     23982.92   
1       010810510      001  3-019          0.50           10     76355.41   
2       010810510      001  4-073          0.40           20     50295.25   
3       010810510      001  4-053          0.50           20     25570.92   
4       010810510      001  2-006          0.25            5   2027361.50   
5       010810510      001  3-016          0.25           10     53397.09   
6       010810510      001  3-020          0.60           10    148434.35   
7       010810510      001  3-023          0.60           10    153277.38   
8       010810510      001  4-055          0.70           20   1321238.52   
9       010810510      001  4-068          0.50           20     64301.18   
10      010810510      001  4-075          0.60           20     34218.91   
11      010810510      001  4-074          0.60           20    119058.47   
12      010810510      001  4-067          0.70           20     67330.85   
13      010810510      001  4-057          0.50           20     16455.77   
14      010810510      001  3-021          0.50           10    131461.73   
15      010810510      001  4-069          0.50           20     12054.23   
16      010810510      001  3-022          0.40           10     50295.25   
17      010810510      001  4-056          0.50           20     36941.32   
18      010810510      001  3-018          0.60           10     67330.85   
19      010810510      001  4-070          0.70           20    148434.35   
20      010810510      001  3-015          0.60           10   1346809.44   
21      010810510      001  4-072          0.25           20    107478.81   

    perc_covered start_date expiry_date  
0       0.557829 2022-01-25  2039-05-26  
1       0.912109 2022-01-25  2039-05-26  
2       0.771633 2022-01-25  2039-05-26  
3       0.398730 2022-01-25  2039-05-26  
4       0.832291 2022-01-25  2039-05-26  
5       0.470183 2022-01-25  2039-05-26  
6       0.837125 2022-01-25  2039-05-26  
7       0.724995 2022-01-25  2039-05-26  
8       0.909410 2022-01-25  2039-05-26  
9       0.911261 2022-01-25  2039-05-26  
10      0.745471 2022-01-25  2039-05-26  
11      0.719317 2022-01-25  2039-05-26  
12      0.965610 2022-01-25  2039-05-26  
13      0.524623 2022-01-25  2039-05-26  
14      0.664032 2022-01-25  2039-05-26  
15      0.916660 2022-01-25  2039-05-26  
16      0.771633 2022-01-25  2039-05-26  
17      0.449409 2022-01-25  2039-05-26  
18      0.965610 2022-01-25  2039-05-26  
19      0.837125 2022-01-25  2039-05-26  
20      0.887821 2022-01-25  2039-05-26  
21      0.693493 2022-01-25  2039-05-26

In [37]:
def eval(row):
    start = row['start_date']
    
    if (start.replace(year = start.year + row['term_length']) - datetime.datetime.today()).total_seconds() <= 0:
        return 'True'

    return 'False'

In [38]:
res['condition_met'] = res.apply(lambda row: True if row['perc_covered'] >= row['min_coverage'] else False, axis=1)
res['term_expired'] = res.apply(lambda row: eval(row), axis=1)
res

license_number revision   area  min_coverage  term_length  pop_covered  \
0       010810510      001  4-071          0.25           20     23982.92   
1       010810510      001  3-019          0.50           10     76355.41   
2       010810510      001  4-073          0.40           20     50295.25   
3       010810510      001  4-053          0.50           20     25570.92   
4       010810510      001  2-006          0.25            5   2027361.50   
5       010810510      001  3-016          0.25           10     53397.09   
6       010810510      001  3-020          0.60           10    148434.35   
7       010810510      001  3-023          0.60           10    153277.38   
8       010810510      001  4-055          0.70           20   1321238.52   
9       010810510      001  4-068          0.50           20     64301.18   
10      010810510      001  4-075          0.60           20     34218.91   
11      010810510      001  4-074          0.60           20    119058.47   
12      010810510      001  4-067          0.70           20     67330.85   
13      010810510      001  4-057          0.50           20     16455.77   
14      010810510      001  3-021          0.50           10    131461.73   
15      010810510      001  4-069          0.50           20     12054.23   
16      010810510      001  3-022          0.40           10     50295.25   
17      010810510      001  4-056          0.50           20     36941.32   
18      010810510      001  3-018          0.60           10     67330.85   
19      010810510      001  4-070          0.70           20    148434.35   
20      010810510      001  3-015          0.60           10   1346809.44   
21      010810510      001  4-072          0.25           20    107478.81   

    perc_covered start_date expiry_date  condition_met term_expired  
0       0.557829 2022-01-25  2039-05-26           True        False  
1       0.912109 2022-01-25  2039-05-26           True        False  
2       0.771633 2022-01-25  2039-05-26           True        False  
3       0.398730 2022-01-25  2039-05-26          False        False  
4       0.832291 2022-01-25  2039-05-26           True        False  
5       0.470183 2022-01-25  2039-05-26           True        False  
6       0.837125 2022-01-25  2039-05-26           True        False  
7       0.724995 2022-01-25  2039-05-26           True        False  
8       0.909410 2022-01-25  2039-05-26           True        False  
9       0.911261 2022-01-25  2039-05-26           True        False  
10      0.745471 2022-01-25  2039-05-26           True        False  
11      0.719317 2022-01-25  2039-05-26           True        False  
12      0.965610 2022-01-25  2039-05-26           True        False  
13      0.524623 2022-01-25  2039-05-26           True        False  
14      0.664032 2022-01-25  2039-05-26           True        False  
15      0.916660 2022-01-25  2039-05-26           True        False  
16      0.771633 2022-01-25  2039-05-26           True        False  
17      0.449409 2022-01-25  2039-05-26          False        False  
18      0.965610 2022-01-25  2039-05-26           True        False  
19      0.837125 2022-01-25  2039-05-26           True        False  
20      0.887821 2022-01-25  2039-05-26           True        False  
21      0.693493 2022-01-25  2039-05-26           True        False